In [21]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import random
from array import array
from collections import Counter
import operator
import time

from random import randrange  
  
from sklearn.datasets import make_blobs  
from sklearn.preprocessing import normalize 

from math import log
from collections import Counter
from random import shuffle

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


def calcShannonEnt(dataSet):  # Calculate the entropy of the data
    numEntries=len(dataSet)  # len of data
    labelCounts={}
    for featVec in dataSet:
        currentLabel=featVec[-1] # The last word (category) of each row of data
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1  # Count number of classes there are and the number of each class
    shannonEnt=0
    for key in labelCounts:
        prob=float(labelCounts[key])/numEntries # Calculate the entropy of a single class
        shannonEnt-=prob*log(prob,2) # Accumulate the entropy of each class
    return shannonEnt

def splitDataSet(dataSet,axis,value): # Split data by a certain feature (best feature)
    retDataSet=[]
    for featVec in dataSet:
        if featVec[axis]==value:
            reducedFeatVec =featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

def chooseBestFeatureToSplit(dataSet):  # Choose the best classification features
    numFeatures = len(dataSet[0])-1
    baseEntropy = calcShannonEnt(dataSet)  # Original entropy
    bestInfoGain = 0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prob =len(subDataSet)/float(len(dataSet))
            newEntropy +=prob*calcShannonEnt(subDataSet)  # Entropy after classification
        infoGain = baseEntropy - newEntropy  # Difference between the original entropy and the entropy classified by feature
        
        if (infoGain>bestInfoGain):   # If after dividing by a certain feature, the entropy value is reduced the most, then the secondary feature is the optimal classification feature
            bestInfoGain=infoGain
            bestFeature = i
    return bestFeature

def cal_feature_weight22(data):
  #X,Y=convert_data2(data)
  df = pd.DataFrame(data)

  #cal_feature_weight(data)

  label=df.iloc[:,-1:]

  df.drop(columns=df.columns[-1], axis=1,inplace=True)
  #convert data after delete label column to numpy array
  X=df.to_numpy()

  #convert the label column to a row
  label=label.transpose()

  #convert label to a matrix
  Y=np.concatenate((label.to_numpy()))
  n_sample,n_feature=np.shape(X)

  my_label_vector = Y
  my_input_matrix=X

  r = relief.Relief(n_features=n_feature)

  my_transformed_matrix = r.fit_transform(my_input_matrix,my_label_vector)

  weight=r.w_

  #return np.delete(weight, -1)

  return weight

def choose_the_feat22(data):
  idx=np.argmax(cal_feature_weight22(data))
  return idx

def createSubtable(dataTable, index, value):
    '''
    returns sub-dataTable based on the given dataTable, index of the attribute and its value.
    '''

    subDataTable = []
    for row in dataTable:
        if row[index] == value:
            chopped_row = row[:index]  # setting the values that are BEFORE the index
            chopped_row.extend(row[index+1:])  # setting the value that are AFTER the index
            subDataTable.append(chopped_row) # adding both of them, eventually returning reduced dataTable.
    return subDataTable
    
def majority_voting(decision):
    '''
    returns the most popular element from the list given
    '''

    return Counter(decision).most_common()[0][1]


def C45_chooseBestFeatureToSplit(dataset):
    numFeatures=len(dataset[0])-1
    baseEnt=calcShannonEnt(dataset)
    bestInfoGain_ratio=0.0
    bestFeature=-1
    for i in range(numFeatures): #Traverse all features
        featList=[example[i]for example in dataset]  
        uniqueVals=set(featList) #The feature list is created as a set collection, and the elements cannot be repeated. Create a unique category label list
        newEnt=0.0
        IV=0.0
        for value in uniqueVals:     #Calculate the information entropy of each division method
            subdataset=splitDataSet(dataset,i,value)
            p=len(subdataset)/float(len(dataset))
            newEnt+=p*calcShannonEnt(subdataset)
            IV=IV-p*log(p,2)
        infoGain=baseEnt-newEnt
        if (IV == 0): 
            continue
        infoGain_ratio = infoGain / IV                    
        
        if (infoGain_ratio >bestInfoGain_ratio):          #Select the largest gain ratio
            bestInfoGain_ratio = infoGain_ratio
            bestFeature = i                              #Select the feature corresponding to the largest gain ratio
    return bestFeature

def CART_chooseBestFeatureToSplit(dataset):

    numFeatures = len(dataset[0]) - 1
    bestGini = 999999.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)
        gini = 0.0
        for value in uniqueVals:
            subdataset=splitDataSet(dataset,i,value)
            p=len(subdataset)/float(len(dataset))
            subp = len(splitDataSet(subdataset, -1, '0')) / float(len(subdataset))
        gini += p * (1.0 - pow(subp, 2) - pow(1 - subp, 2))
        
        if (gini < bestGini):
            bestGini = gini
            bestFeature = i
    return bestFeature

def majorityCnt(classList):    # Sort by the number of categories after classification, for example: the final classification is 2 male and 1 female, then it is judged as male;
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

def ID3_createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]  
    if classList.count(classList[0])==len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    bestFeat=chooseBestFeatureToSplit(dataSet) 
    bestFeatLabel=labels[bestFeat]
   
    myTree={bestFeatLabel:{}} #create the dict
    del(labels[bestFeat])
    featValues=[example[bestFeat] for example in dataSet]
    uniqueVals=set(featValues)
    for value in uniqueVals:
        subLabels=labels[:]
        myTree[bestFeatLabel][value]=ID3_createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

def C45_createTree(dataset,labels):
    classList=[example[-1] for example in dataset]
    if classList.count(classList[0]) == len(classList):
        # The categories are exactly the same, stop dividing
        return classList[0]
    if len(dataset[0]) == 1:
        # After traversing all features, return the most frequent occurrence
        return majorityCnt(classList)
    bestFeat = C45_chooseBestFeatureToSplit(dataset)
    bestFeatLabel = labels[bestFeat]
    
    C45Tree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    # Get the list including all the attribute values of the node
    featValues = [example[bestFeat] for example in dataset]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        C45Tree[bestFeatLabel][value] = C45_createTree(splitDataSet(dataset, bestFeat, value), subLabels)
    return C45Tree

def CART_createTree(dataset,labels):
    classList=[example[-1] for example in dataset]
    if classList.count(classList[0]) == len(classList):
        # The categories are exactly the same, stop dividing
        return classList[0]
    if len(dataset[0]) == 1:
        # After traversing all features, return the most frequent occurrence
        return majorityCnt(classList)
    bestFeat = CART_chooseBestFeatureToSplit(dataset)
    
    bestFeatLabel = labels[bestFeat]
    
    CARTTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    # Get the list including all the attribute values of the node
    featValues = [example[bestFeat] for example in dataset]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        CARTTree[bestFeatLabel][value] = CART_createTree(splitDataSet(dataset, bestFeat, value), subLabels)
    return CARTTree

def FWDT_createTree(dataTable, labels):
    '''
    Generates tree based on the dataTable given. dataTable shrinks over time which allows recursive algorithm.
    '''
    #X,Y=convert_data(dataTable)

    decision = [row[-1] for row in dataTable]

    '''
    dataCounter = Counter(decision) #to find the most common class
    majorityClass = dataCounter.most_common(1)[0][0] #set the value to majorityClass

    dataCounter = Counter(Y) #to find the most common class
    majorityClass = dataCounter.most_common(1)[0][0] #set the value to majorityClass

    if Y.count(Y[0]) == len(Y):
        return Y[0]              # return when all of the decision in the dataTable is same

    if len(X[0]) == 1:
        return majority_voting(Y)    # do the majority voting if there is only one remaining feature in dataTable
    '''

    dataCounter = Counter(decision) #to find the most common class
    majorityClass = dataCounter.most_common(1)[0][0] #set the value to majorityClass

    if decision.count(decision[0]) == len(decision):
        return decision[0]              # return when all of the decision in the dataTable is same

    if len(dataTable[0]) == 1:
        return majority_voting(decision)    # do the majority voting if there is only one remaining feature in dataTable

    #root_attribute = NodeSelection(dataTable)
    root_attribute = choose_the_feat22(dataTable)

    root_attributeLabel = labels[root_attribute]
    tree = {root_attributeLabel: {}}

    del (labels[root_attribute])  # reducing the dataTable so the recursion will work
    attributeValuesAll = [row[root_attribute] for row in dataTable]
    attribute_values = set(attributeValuesAll)  # finding the unique values

    #attribute_values.add(None)
    for value in attribute_values:
        #if value == None:
           # tree[root_attributeLabel][value] = majorityClass #set the None branch to majority class
      sub_labels = labels[:]
      tree[root_attributeLabel][value] = FWDT_createTree(createSubtable(dataTable, root_attribute, value), sub_labels)
    
    return tree

In [22]:
pip install sklearn_relief

In [23]:
import sklearn_relief as relief
#from ReliefF import ReliefF

In [24]:
def convert_data_all(data,features):
  new_data=pd.DataFrame()
  for feature in features:
    col=data[feature]
    if (str(list(col)[0]).split(".")[0]).isdigit() or str(list(col)[0]).isdigit() or (str(list(col)[0]).split('-')[-1]).split(".")[-1].isdigit():
      new_data[feature]=data[feature]
    else:
      keys=list(set(list(col)))
      values=list(range(len(keys)))
      new=dict(zip(keys,values))
      new_data[feature]=data[feature].map(new)
    #new_data[features[-1]]=data[features[-1]]
  return new_data
  
def predict(query,tree,default = 1):
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result


def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    #print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')
    return predicted["predicted"], data["class"]


In [25]:
def createDataTableCsv(data):
    '''
    Takes data in form of pandas dataFrame that was extracted from .csv file. Note that the labels are manually
    typed in. As long as the data from .csv can return list of dataTable and list of dataLabels, the algorithm should work.
    '''

    global dataTable_testing
    dataTable = data.values.tolist()  # Use .values to get a numpy.array and then .tolist() to get a list.
    shuffle(dataTable)
    training_size = int(len(dataTable) * 0.8)  # 90 percent of the sample after the randomization.
    dataTable_training = list(dataTable[0:training_size])
    dataTable_testing = list(dataTable[training_size:])

    return dataTable_training,dataTable_testing

#TEST OF DATASET

In [26]:
def testModel(testData):
  print(testData)
  #import data
  data=pd.read_csv(testData)
  data=convert_data_all(data,list(data.columns))

  dataTable_training,dataTable_testing= createDataTableCsv(data)
  treeID3 = ID3_createTree(dataTable_training, list(data.columns))
  treeC45 = C45_createTree(dataTable_training, list(data.columns))
  treeCART = CART_createTree(dataTable_training, list(data.columns))

  treeFWDT = FWDT_createTree(dataTable_training, list(data.columns))
  #test(testing_data,tree)

  pandas_df = pd.DataFrame(dataTable_testing)
  pandas_df.columns=list(data.columns)

  y_predID3,y_testID3=test(pandas_df,treeID3)
  y_predC45,y_testC45=test(pandas_df,treeC45)
  y_predCART,y_testCART=test(pandas_df,treeCART)

  y_predFWDT,y_testFWDT=test(pandas_df,treeFWDT)

  #Fill in the missing value if any by median method
  y_predID3 = y_predID3.fillna(lambda x: x.median())
  y_testID3=y_testID3.fillna(lambda x: x.median())
  y_predC45 = y_predC45.fillna(lambda x: x.median())
  y_testC45=y_testC45.fillna(lambda x: x.median())
  y_predCART = y_predCART.fillna(lambda x: x.median())
  y_testCART=y_testCART.fillna(lambda x: x.median())

  y_predFWDT = y_predFWDT.fillna(lambda x: x.median())
  y_testFWDT=y_testFWDT.fillna(lambda x: x.median())

  #calculate accurancy:
  '''
  print('Accurancy of CART tree: ',round((np.sum(y_testCART == y_predCART)/len(y_predCART))*100),3)
  print('Accurancy of ID3 tree: ',round((np.sum(y_testID3 == y_predID3)/len(y_predID3))*100),3)
  print('Accurancy of C45 tree: ',round((np.sum(y_testC45 == y_predC45)/len(y_predC45))*100),3)
  print('Accurancy of FWDT tree: ',round((np.sum(y_testFWDT == y_predFWDT)/len(y_predFWDT))*100),3)
  '''

  '''
  classification_report
  '''

  #ID3
  y_tID3=y_testID3.values
  y_pID3=y_predID3.values

  y_tID3=y_tID3.astype(np.float)
  y_pID3=y_pID3.astype(np.float)
  print('ID3: ')
  print("Accurancy, Recall; F1-score: %s, %s, %s"% (round((np.sum(y_testID3 == y_predID3)/len(y_predID3))*100,3),
        round(recall_score(y_tID3, y_pID3, average="macro")*100,3),
        round(f1_score(y_tID3, y_pID3, average="macro")*100,3)))
  print('\n')


  #C45
  y_tC45=y_testC45.values
  y_pC45=y_predC45.values

  y_tC45=y_tC45.astype(np.float)
  y_pC45=y_pC45.astype(np.float)
  print('C45: ')
  print("Accurancy, Recall; F1-score: %s, %s, %s"% (round((np.sum(y_testC45 == y_predC45)/len(y_predC45))*100,3),
        round(recall_score(y_tC45, y_pC45, average="macro")*100,3),
        round(f1_score(y_tC45, y_pC45, average="macro")*100,3)))
  print('\n')


  #CART
  y_tCART=y_testCART.values
  y_pCART=y_predCART.values

  y_tCART=y_tCART.astype(np.float)
  y_pCART=y_pCART.astype(np.float)
  print('CART: ')
  print("Accurancy, Recall; F1-score: %s, %s, %s"% (round((np.sum(y_testCART == y_predCART)/len(y_predCART))*100,3),
        round(recall_score(y_tCART, y_pCART, average="macro")*100,3),
        round(f1_score(y_tCART, y_pCART, average="macro")*100,3)))
  print('\n')
  
  #FWDT
  y_tFWDT=y_testFWDT.values
  y_pFWDT=y_predFWDT.values

  y_tFWDT=y_tFWDT.astype(np.float)
  y_pFWDT=y_pFWDT.astype(np.float)
  print('FWDT: ')
  print("Accurancy, Recall; F1-score: %s, %s, %s"% (round((np.sum(y_testFWDT == y_predFWDT)/len(y_predFWDT))*100,3),
        round(recall_score(y_tFWDT, y_pFWDT, average="macro")*100,3),
        round(f1_score(y_tFWDT, y_pFWDT, average="macro")*100,3)))
  print('\n')


In [27]:
listData0=['nurseryf.csv','breast_cancerf.csv','car_evaluationf.csv',
      'mushroomf.csv',
      'spectf.csv','statlogff.csv','zoof.csv','Immunotherapy.csv','app_recordf.csv']

listData=['nurseryf_filtered.csv','breast_cancerf_filtered.csv','car_evaluationf_filtered.csv',
      'mushroomf_filtered.csv',
      'spectf_filtered.csv','statlogff_filtered.csv','zoof_filtered.csv','Immunotherapy_filtered.csv','app_recordf_filtered.csv']

listDataq1=['nurseryf_filtered_q1.csv','breast_cancerf_filtered_q1.csv','car_evaluationf_filtered_q1.csv',
      'mushroomf_filtered_q1.csv',
      'spectf_filtered_q1.csv','statlogff_filtered_q1.csv','zoof_filtered_q1.csv','Immunotherapy_filtered_q1.csv','app_recordf_filtered_q1.csv']

In [ ]:
for i in range(len(listData0)):

 testModel(listData0[i]),testModel(listData[i]),testModel(listDataq1[i])

In [30]:
pip install ReliefF

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for ReliefF: filename=ReliefF-0.1.2-cp37-none-any.whl size=6319 sha256=24c39d323f2a31dcd0a232c4175883ae4055ac6e58e5bffecd5bdedcc3adbd43
  Stored in directory: /root/.cache/pip/wheels/1c/9e/51/59ca638520794c9e0155d592b9f8c579f80cc29cbaf1de0f45
Successfully built ReliefF


In [32]:
from ReliefF import ReliefF
import numpy as np
from sklearn import datasets
import pandas as pd

#example of 2 class problem
data = np.array([[9,2,2],[5,1,0],[9,3,2],[8,3,1],[6,0,0]])
target = np.array([0,0,1,1,1])

fs = ReliefF(n_neighbors=1, n_features_to_keep=2)

print (fs.w_)

AttributeError: ignored